In [17]:
from mat2vec.processing import MaterialsTextProcessor
from os import path
from scipy import spatial

from chemdataextractor.doc import Document, Heading, Paragraph
from chemdataextractor.nlp.tokenize import ChemWordTokenizer

from gensim.models import Word2Vec
from gensim.models import word2vec
from gensim.models.word2vec import LineSentence

from sklearn import metrics
from sklearn.cluster import DBSCAN
from sklearn.manifold import TSNE

import matplotlib.pyplot as plt
import pandas as pd
import regex

In [18]:
Path = './Project/调试11/'

# 预处理

In [20]:
Data =pd.read_csv(Path + 'Data.txt', header  = None, names = ['Abstract'])
Df = pd.DataFrame(Data['Abstract'])
Lists_Data = Df.values.tolist()
Data.head()

,Abstract
0,This work proposes an improvement of gold nano...
1,Carbon monoxide has been recognized relatively...
2,Two luminescent coordination polymers were con...
3,We have identified a ligand (CO) accommodation...
4,Electronic sensors capable of capturing mechan...


In [22]:
ELEMENTS = ["H", "He", "Li", "Be", "B", "C", "N", "O", "F", "Ne", "Na", "Mg", "Al", "Si", "P", "S", "Cl", "Ar", "K",
                "Ca", "Sc", "Ti", "V", "Cr", "Mn", "Fe", "Co", "Ni", "Cu", "Zn", "Ga", "Ge", "As", "Se", "Br", "Kr",
                "Rb", "Sr", "Y", "Zr", "Nb", "Mo", "Tc", "Ru", "Rh", "Pd", "Ag", "Cd", "In", "Sn", "Sb", "Te", "I",
                "Xe", "Cs", "Ba", "La", "Ce", "Pr", "Nd", "Pm", "Sm", "Eu", "Gd", "Tb", "Dy", "Ho", "Er", "Tm", "Yb",
                "Lu", "Hf", "Ta", "W", "Re", "Os", "Ir", "Pt", "Au", "Hg", "Tl", "Pb", "Bi", "Po", "At", "Rn", "Fr",
                "Ra", "Ac", "Th", "Pa", "U", "Np", "Pu", "Am", "Cm", "Bk", "Cf", "Es", "Fm", "Md", "No", "Lr", "Rf",
                "Db", "Sg", "Bh", "Hs", "Mt", "Ds", "Rg", "Cn", "Nh", "Fl", "Mc", "Lv", "Ts", "Og", "Uue"]
#ELEMENTS
ELEMENTS_LOW = []
for ELEMENT in ELEMENTS:
    ELEMENT_LOW = ELEMENT.lower()
    ELEMENTS_LOW.append(ELEMENT_LOW)
ELEMENTS_LOW_ION = regex.compile(r"^("+r"|".join(ELEMENTS_LOW) + r")(\(?\d*[+-]\)?)$")

In [23]:
F_Data_handle = open(Path + 'Data_Phrases.txt', 'w', encoding='utf-8')
text_processor = MaterialsTextProcessor()

processed = []
for List_Data in Lists_Data:
    List_Data_str = ''.join('%s' %i for i in List_Data)
    Paras = Paragraph(List_Data_str)
    for Sentence in Paras:
        Sentence_handel = text_processor.process(str(Sentence),normalize_materials=False)
        Tokens = Sentence_handel[0]
        for Token in Tokens:
            if ELEMENTS_LOW_ION.match(Token) is not None:
                Token = Token.capitalize()
            processed.append(Token)

Sentence_phrases = text_processor.make_phrases(processed,reps=1)
for Token in Sentence_phrases:
    F_Data_handle.write(str(Token) + ' ')
F_Data_handle.close()

# 提取化学式

In [16]:
F_Data_handle = open(Path + 'Data_Phrases.txt', 'r', encoding = 'utf-8')
F_Cems = open(Path + 'Cems.txt', 'w', encoding = 'utf-8')
F_Tmps = open(Path + 'Tmp.txt', 'w', encoding = 'utf-8')

Cems_handle = []

Data_handle = ''.join('%s' %i for i in F_Data_handle)
Sentences = Paragraph(Data_handle)
Cems = Sentences.cems
for Cem in Cems:
    F_Cems.write(str(Cem) + '\n')
F_Cems.close()

F_Cems = open(Path + 'Cems.txt', 'r', encoding = 'utf-8')
Cems_tmp = []
for Lines in F_Cems:
    Lines_handle = Lines.strip('\n').split('/')
    for Line_handle in Lines_handle:
        if not Line_handle in Cems_tmp:
            Cems_tmp.append(Line_handle)
            F_Tmps.write(Line_handle + '\n')
F_Data_handle.close()

F_Tmps.close()

# 训练word2vec模型

In [8]:
Data_word2vec = word2vec.LineSentence(Path + 'Data_Phrases.txt')
Model = Word2Vec(Data_word2vec, vector_size = 200, sg=1, window = 8, negative = 15, alpha = 0.01,epochs = 5, min_count =1)
Model.save(Path + 'wod2vec.model')

In [31]:
Model = Word2Vec.load(Path + 'wod2vec.model')

words = Model.wv.index_to_key
#print(words)

tmp = [val for val in Cems_tmp if val in words]
#print(tmp)

In [33]:
T = []
Y = []

for x in tmp:
    y = Model.wv.similarity(u"gas",x)
    T.append(str(x))
    Y.append(str(y))
df = pd.DataFrame({'化学式':T,'相关度':Y})
df.to_csv(Path + 'gas.csv',index=False,sep=',')

In [ ]:
gas_sensor